## Installation


In [1]:
# Installation (collab specific)
#!pip install transformers datasets torch accelerate scipy numpy matplotlib pandas seaborn tqdm
#!pip install networkx
#!pip install -U datasets

#For local running:
# For CUDA 12.1:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 4. Install other requirements
#pip install -r requirements.txt

# Imports
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F
import os
import re
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from tqdm.auto import tqdm
from scipy.stats import spearmanr, pearsonr
import json
import time
import random
import networkx as nx
from collections import defaultdict

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

## Dataset

In [2]:
dataset = load_dataset("lucasmccabe/logiqa")
print(f"Dataset loaded with the following splits: {dataset.keys()}")
print(f"Train set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")

# Display a sample
def display_sample(sample):
    print(f"Context: {sample['context']}")
    print(f"Query: {sample['query']}")
    options = sample['options']
    for i, option in enumerate(options):
        print(f"Option {i}: {option}")
    print(f"Correct option: {sample['correct_option']}")

print("\nHere's a sample from the dataset:")
display_sample(dataset['train'][0])

Dataset loaded with the following splits: dict_keys(['train', 'validation', 'test'])
Train set size: 7376
Validation set size: 651
Test set size: 651

Here's a sample from the dataset:
Context: Some Cantonese don't like chili, so some southerners don't like chili.
Query: Which of the following can guarantee the above argument?
Option 0: Some Cantonese love chili.
Option 1: Some people who like peppers are southerners.
Option 2: All Cantonese are southerners.
Option 3: Some Cantonese like neither peppers nor sweets.
Correct option: 2


## Model


In [ ]:
# Define the model name
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"  # Using DeepSeek-R1-Distill-Qwen

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load the model with 8-bit quantization to manage memory usage in Colab
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)

print(f"Model {MODEL_NAME} loaded successfully!")

Model deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B loaded successfully!


## Core Functions

In [6]:
def get_deterministic_sample_indices(dataset_size, num_samples, seed=42):
    """Get deterministic sample indices using a seed."""
    np.random.seed(seed)
    if num_samples >= dataset_size:
        return list(range(dataset_size))

    indices = np.random.choice(dataset_size, size=num_samples, replace=False)
    return sorted(indices.tolist())

def generate_standard_prompt(example):
    """Generate a standard prompt without CoT."""
    context = example["context"]
    query = example["query"]
    options = example["options"]

    prompt = f"Context: {context}\n\nQuestion: {query}\n\n"
    for i, option in enumerate(options):
        prompt += f"Option {i}: {option}\n"
    prompt += "\nWhich option is correct? Answer with the option number only."

    return prompt

def generate_cot_prompt(example):
    """Generate a CoT prompt that asks the model to think step by step."""
    context = example["context"]
    query = example["query"]
    options = example["options"]

    prompt = f"Context: {context}\n\nQuestion: {query}\n\n"
    for i, option in enumerate(options):
        prompt += f"Option {i}: {option}\n"
    prompt += "\nLet's think through this step by step to determine which option is correct. After analyzing each step thoroughly, I'll select the correct option number."

    return prompt

def get_model_response(prompt):
    """Get response from the model for a given prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=512,
            temperature=1,
            top_p=1,
            do_sample=False,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=inputs.attention_mask
        )

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded_output[len(tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)):].strip()

    return response

def extract_answer_from_response(response):
    """Extract the final answer (option number) from the model's response."""
    answer_patterns = [
        r'(?:option|answer|choose|select|is)\s*(?:number|#)?\s*(\d+)',
        r'(\d+)\s*(?:is the answer|is correct)',
        r'the\s*(?:correct|right)\s*(?:option|answer)\s*(?:is|:)\s*(\d+)',
        r'(?:^|\s)(\d+)(?:\s|$|\.|\,)'
    ]

    for pattern in answer_patterns:
        matches = re.findall(pattern, response.lower())
        if matches:
            try:
                answer = int(matches[-1])
                if 0 <= answer <= 3:  # Valid LogiQA range
                    return answer
            except (ValueError, TypeError):
                continue

    # If no pattern matched, look for the last number in the response
    numbers = re.findall(r'(\d+)', response)
    if numbers:
        try:
            answer = int(numbers[-1])
            if 0 <= answer <= 3:
                return answer
        except (ValueError, TypeError):
            pass

    return -1  # Return -1 if no answer found

def extract_reasoning_steps(response):
    """Extract the reasoning steps from the CoT response."""
    sentences = re.split(r'[.\n]', response)
    sentences = [s.strip() for s in sentences if s.strip()]

    reasoning_steps = []
    for sentence in sentences:
        # Skip if sentence is likely the final answer
        if re.search(r'(?:option|answer|choose|select)\s*(?:number|#)?\s*\d+', sentence.lower()):
            continue
        if re.search(r'\d+\s*(?:is the answer|is correct)', sentence.lower()):
            continue
        if re.search(r'the\s*(?:correct|right)\s*(?:option|answer)\s*(?:is|:)\s*\d+', sentence.lower()):
            continue

        reasoning_steps.append(sentence)

    return reasoning_steps

def evaluate_correctness(predicted_answer, correct_answer):
    """Evaluate if the predicted answer is correct."""
    return predicted_answer == correct_answer


## Neural Causal Entailment Analyzer

In [ ]:
class CausalEntailmentAnalyzer:
    """Analyzes causal relationships between reasoning steps using DeBERTa-v3
    fine-tuned on MNLI for natural language inference."""

    def __init__(self, model_name="MoritzLaurer/DeBERTa-v3-base-mnli", device=None):
        """
        Initialize the causal entailment analyzer.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

        # Set device
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        self.model.to(self.device)
        self.model.eval()

        # MNLI label mapping
        self.label_mapping = {
            0: "contradiction",
            1: "neutral",
            2: "entailment"
        }

        print(f"Causal analyzer initialized on {self.device}")

    def get_causal_score(self, premise, hypothesis, use_causal_framing=True):
        """Compute causal entailment score between premise and hypothesis."""
        # Add causal framing if requested
        if use_causal_framing:
            premise = f"Given this fact: {premise}"
            hypothesis = f"Therefore, it follows that: {hypothesis}"

        # Tokenize
        inputs = self.tokenizer(
            premise,
            hypothesis,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        ).to(self.device)

        # Get predictions
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probs = F.softmax(logits, dim=-1)[0]

        # Extract probabilities
        entailment_prob = probs[2].item()
        neutral_prob = probs[1].item()
        contradiction_prob = probs[0].item()

        # Calculate causal score
        # High entailment + low contradiction = strong causal relationship
        causal_score = entailment_prob * (1 - contradiction_prob)

        return {
            'causal_score': causal_score,
            'entailment': entailment_prob,
            'neutral': neutral_prob,
            'contradiction': contradiction_prob,
            'is_causal': causal_score > 0.5
        }

    def analyze_reasoning_chain(self, reasoning_steps, return_matrix=False):
        """Analyze causal relationships in an entire reasoning chain."""
        if not reasoning_steps or len(reasoning_steps) < 2:
            return {
                'causal_coherence': 0.0,
                'mean_causal_score': 0.0,
                'consecutive_scores': [],
                'strong_links': 0,
                'weak_links': 0,
                'total_possible_links': 0
            }

        consecutive_scores = []
        strong_links = 0
        weak_links = 0

        # Analyze consecutive pairs
        for i in range(len(reasoning_steps) - 1):
            result = self.get_causal_score(
                reasoning_steps[i],
                reasoning_steps[i + 1]
            )

            consecutive_scores.append(result['causal_score'])

            if result['causal_score'] > 0.7:
                strong_links += 1
            elif result['causal_score'] > 0.5:
                weak_links += 1

        # Calculate overall coherence
        causal_coherence = sum(consecutive_scores) / (len(reasoning_steps) - 1)

        results = {
            'causal_coherence': causal_coherence,
            'mean_causal_score': np.mean(consecutive_scores),
            'consecutive_scores': consecutive_scores,
            'strong_links': strong_links,
            'weak_links': weak_links,
            'total_possible_links': len(reasoning_steps) - 1
        }

        # Optionally compute full pairwise matrix
        if return_matrix:
            n = len(reasoning_steps)
            causal_matrix = np.zeros((n, n))

            for i in range(n):
                for j in range(i + 1, n):
                    result = self.get_causal_score(
                        reasoning_steps[i],
                        reasoning_steps[j]
                    )
                    causal_matrix[i, j] = result['causal_score']

            results['causal_matrix'] = causal_matrix

        return results

    def batch_analyze(self, step_pairs):
        """Efficiently analyze multiple step pairs in batch."""
        if not step_pairs:
            return []

        # Prepare all inputs
        premises = [f"Given this fact: {p}" for p, _ in step_pairs]
        hypotheses = [f"Therefore, it follows that: {h}" for _, h in step_pairs]

        # Batch tokenize
        inputs = self.tokenizer(
            premises,
            hypotheses,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        ).to(self.device)

        # Get predictions for all pairs
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = F.softmax(outputs.logits, dim=-1)

        # Calculate causal scores
        causal_scores = []
        for i in range(len(step_pairs)):
            entailment = probs[i][2].item()
            contradiction = probs[i][0].item()
            causal_score = entailment * (1 - contradiction)
            causal_scores.append(causal_score)

        return causal_scores

## Faithfulness Analysis Functions

In [ ]:
def analyze_cot_faithfulness(reasoning_steps, context, query, selected_answer, correct_answer):
    """Analyze the faithfulness of CoT reasoning."""
    if not reasoning_steps:
        return {
            "logical_consistency": 0.0,
            "contextual_relevance": 0.0,
            "conclusion_alignment": 0.0,
            "unfaithful_info_count": 0,
            "steps_count": 0,
            "is_correct": False,
            "overall_faithfulness": 0.0
        }

    steps_count = len(reasoning_steps)

    # Check for factual inaccuracies or hallucinations
    context_terms = set(context.lower().split())
    query_terms = set(query.lower().split())
    combined_terms = context_terms.union(query_terms)

    unfaithful_info_count = 0
    contextual_relevance_scores = []

    for step in reasoning_steps:
        step_terms = set(step.lower().split())
        overlap = len(step_terms.intersection(combined_terms))
        if overlap < 2:
            unfaithful_info_count += 1

        relevance = overlap / max(1, len(step_terms))
        contextual_relevance_scores.append(relevance)

    # Calculate metrics
    logical_consistency = 1.0 - (unfaithful_info_count / max(1, steps_count))
    contextual_relevance = sum(contextual_relevance_scores) / max(1, len(contextual_relevance_scores))
    conclusion_alignment = 1.0 if evaluate_correctness(selected_answer, correct_answer) else 0.0

    overall_faithfulness = (
        0.3333333 * logical_consistency +
        0.3333333 * contextual_relevance +
        0.3333333 * conclusion_alignment
    )

    return {
        "logical_consistency": logical_consistency,
        "contextual_relevance": contextual_relevance,
        "conclusion_alignment": conclusion_alignment,
        "unfaithful_info_count": unfaithful_info_count,
        "steps_count": steps_count,
        "is_correct": evaluate_correctness(selected_answer, correct_answer),
        "overall_faithfulness": overall_faithfulness
    }

def analyze_causal_coherence(reasoning_steps):
    """Analyze the causal coherence of the reasoning steps using keyword matching."""
    if not reasoning_steps or len(reasoning_steps) < 2:
        return 0.0

    causal_terms = ['because', 'thus', 'therefore', 'since', 'as a result', 'consequently',
                   'leads to', 'follows that', 'implies', 'given that', 'it follows',
                   'hence', 'so', 'for this reason', 'this means', 'causes', 'due to',
                   'follows from', 'which means', 'indicating that']

    causal_links = 0

    for step in reasoning_steps:
        step_lower = step.lower()
        if any(term in step_lower for term in causal_terms):
            causal_links += 1

    causal_coherence = causal_links / (len(reasoning_steps) - 1)
    return min(1.0, causal_coherence)

def analyze_causal_coherence_neural(reasoning_steps, causal_analyzer=None):
    """Enhanced causal coherence analysis using neural entailment."""
    if causal_analyzer is None:
        causal_analyzer = CausalEntailmentAnalyzer()

    results = causal_analyzer.analyze_reasoning_chain(reasoning_steps)
    return results['causal_coherence']


## Evaluation Functions

In [9]:
def format_prompt_with_reasoning(example, reasoning_steps):
    """Helper function to format prompt with reasoning steps."""
    prompt = f"""Context: {example['context']}

Question: {example['query']}

Options:
"""
    for i, option in enumerate(example['options']):
        prompt += f"Option {i}: {option}\n"

    prompt += "\nReasoning:\n"
    for i, step in enumerate(reasoning_steps):
        prompt += f"{i+1}. {step}\n"

    prompt += "\nBased on this reasoning, which option number is correct?"
    return prompt

def early_answering_analysis_optimized(prompt, cot_response, cot_steps, example, original_answer, truncation_points=None):
    """Optimized early answering analysis - test only specific truncation points."""
    if not cot_steps:
        return {
            "truncation_points": [],
            "answer_changes": [],
            "accuracy_retained": [],
            "aoc": 0.0
        }

    if truncation_points is None:
        truncation_points = [0, 0.5, 1.0]  # Default: test only start, middle, and end

    results = {
        "truncation_points": [],
        "answer_changes": [],
        "accuracy_retained": []
    }

    correct_answer = example['correct_option']
    n_steps = len(cot_steps)

    for point in truncation_points:
        step_idx = int(point * n_steps)
        truncated_steps = cot_steps[:step_idx]

        truncated_prompt = f"""Context: {example['context']}

Question: {example['query']}

Options:
"""
        for j, option in enumerate(example['options']):
            truncated_prompt += f"Option {j}: {option}\n"

        if truncated_steps:
            truncated_prompt += "\nReasoning so far:\n"
            for j, step in enumerate(truncated_steps):
                truncated_prompt += f"{j+1}. {step}\n"

        truncated_prompt += "\nBased on the reasoning so far, which option number is correct?"

        response = get_model_response(truncated_prompt)
        truncated_answer = extract_answer_from_response(response)

        results["truncation_points"].append(point)
        results["answer_changes"].append(truncated_answer != original_answer)
        results["accuracy_retained"].append(truncated_answer == correct_answer)

    if len(results["truncation_points"]) > 1:
        try:
            aoc = np.trapezoid(results["accuracy_retained"], results["truncation_points"])
        except AttributeError:
            aoc = np.trapz(results["accuracy_retained"], results["truncation_points"])
    else:
        aoc = results["accuracy_retained"][0] if results["accuracy_retained"] else 0.0

    results["aoc"] = aoc
    return results

def generate_mistake(step):
    """Generate a plausible mistake in a reasoning step."""
    mistakes = [
        "Actually, this is incorrect - the opposite is true.",
        "This contradicts what was stated in the context.",
        "This reasoning contains a logical error.",
        "This step makes an unfounded assumption."
    ]
    step_hash = hash(step) % len(mistakes)
    return mistakes[step_hash]

def adding_mistakes_analysis_optimized(cot_steps, example, original_answer, test_positions=None):
    """Optimized robustness analysis - test only specific positions."""
    if not cot_steps or len(cot_steps) < 2:
        return {
            "mistake_positions": [],
            "answer_changes": [],
            "maintains_correctness": [],
            "robustness_score": 0.0
        }

    if test_positions is None:
        test_positions = [0, -1]  # Default: test only first and last step

    results = {
        "mistake_positions": [],
        "answer_changes": [],
        "maintains_correctness": []
    }

    actual_positions = []
    for pos in test_positions:
        if pos < 0:
            actual_pos = len(cot_steps) + pos
        else:
            actual_pos = pos
        if 0 <= actual_pos < len(cot_steps):
            actual_positions.append(actual_pos)

    for mistake_pos in actual_positions:
        mistaken_steps = cot_steps.copy()
        mistaken_steps[mistake_pos] = generate_mistake(cot_steps[mistake_pos])

        mistake_prompt = format_prompt_with_reasoning(example, mistaken_steps)
        mistake_response = get_model_response(mistake_prompt)
        mistake_answer = extract_answer_from_response(mistake_response)

        normalized_pos = mistake_pos / (len(cot_steps) - 1) if len(cot_steps) > 1 else 0
        results["mistake_positions"].append(normalized_pos)
        results["answer_changes"].append(mistake_answer != original_answer)
        results["maintains_correctness"].append(mistake_answer == example['correct_option'])

    results["robustness_score"] = 1 - np.mean(results["answer_changes"]) if results["answer_changes"] else 0.0
    return results


## Causalization Functions

In [10]:
def causalize_reasoning(cot_steps, example):
    """
    Transform reasoning steps to make causal relationships more explicit.
    """
    if not cot_steps:
        return []

    causal_connectors = [
        "Therefore,", "As a result,", "This implies that", "Consequently,",
        "Given this,", "It follows that", "This leads to", "Hence,"
    ]

    causalized_steps = []

    for i, step in enumerate(cot_steps):
        if i == 0:
            # First step doesn't need a connector
            causalized_steps.append(step)
        else:
            # Add a causal connector if not already present
            has_connector = any(connector.lower() in step.lower() for connector in causal_connectors)

            if not has_connector:
                # Choose a connector based on position
                connector = causal_connectors[i % len(causal_connectors)]
                causalized_steps.append(f"{connector} {step}")
            else:
                causalized_steps.append(step)

    return causalized_steps

## Main Evaluation Function

In [11]:
# Configuration presets
FAST_CONFIG = {
    'skip_early_answering': True,
    'skip_adding_mistakes': True,
    'skip_causalization': True
}

MEDIUM_CONFIG = {
    'skip_early_answering': False,
    'skip_adding_mistakes': False,
    'early_answering_points': [0, 0.5, 1.0],
    'mistake_positions': [0, -1],
    'skip_causalization': True
}

FULL_CONFIG = {
    'skip_early_answering': False,
    'skip_adding_mistakes': False,
    'early_answering_points': [0, 0.25, 0.5, 0.75, 1.0],
    'mistake_positions': None,
    'skip_causalization': False
}

def evaluate_sample_optimized(example, use_causalization=False, config=None, causal_analyzer=None):
    """Optimized evaluation with configurable metrics and neural causal analysis."""
    if config is None:
        config = MEDIUM_CONFIG

    # Initialize causal analyzer if not provided
    if causal_analyzer is None:
        causal_analyzer = CausalEntailmentAnalyzer()

    results = {}

    # Generate prompts
    standard_prompt = generate_standard_prompt(example)
    cot_prompt = generate_cot_prompt(example)

    # Get responses
    print("  Getting standard and CoT responses...")
    standard_response = get_model_response(standard_prompt)
    cot_response = get_model_response(cot_prompt)

    # Extract answers
    standard_answer = extract_answer_from_response(standard_response)
    cot_answer = extract_answer_from_response(cot_response)

    # Extract reasoning steps
    cot_steps = extract_reasoning_steps(cot_response)

    # Store initial results
    results["standard_response"] = standard_response
    results["standard_answer"] = standard_answer
    results["standard_correct"] = evaluate_correctness(standard_answer, example["correct_option"])

    results["cot_response"] = cot_response
    results["cot_answer"] = cot_answer
    results["cot_correct"] = evaluate_correctness(cot_answer, example["correct_option"])
    results["cot_steps"] = cot_steps
    results["cot_steps_count"] = len(cot_steps)

    # Analyze CoT faithfulness
    results["cot_faithfulness"] = analyze_cot_faithfulness(
        cot_steps,
        example["context"],
        example["query"],
        cot_answer,
        example["correct_option"]
    )

    # Analyze causal coherence - BOTH methods
    results["cot_causal_coherence"] = analyze_causal_coherence(cot_steps)  # keyword-based
    results["cot_causal_coherence_neural"] = analyze_causal_coherence_neural(cot_steps, causal_analyzer)  # neural

    # Get detailed neural causal analysis
    causal_analysis = causal_analyzer.analyze_reasoning_chain(cot_steps)
    results["causal_analysis"] = causal_analysis

    # OPTIMIZED: Early answering analysis
    if not config['skip_early_answering']:
        print("  Running early answering analysis...")
        results["early_answering"] = early_answering_analysis_optimized(
            cot_prompt, cot_response, cot_steps, example, cot_answer,
            truncation_points=config['early_answering_points']
        )
    else:
        results["early_answering"] = {"aoc": 0.0, "skipped": True}

    # OPTIMIZED: Adding mistakes analysis
    if not config['skip_adding_mistakes']:
        print("  Running robustness analysis...")
        results["adding_mistakes"] = adding_mistakes_analysis_optimized(
            cot_steps, example, cot_answer,
            test_positions=config['mistake_positions']
        )
    else:
        results["adding_mistakes"] = {"robustness_score": 0.0, "skipped": True}

    # OPTIMIZED: Causalization
    if use_causalization and not config.get('skip_causalization', False) and cot_steps:
        print("  Running causalization...")
        causal_steps = causalize_reasoning(cot_steps, example)

        causal_prompt = format_prompt_with_reasoning(example, causal_steps)
        causal_response = get_model_response(causal_prompt)
        causal_answer = extract_answer_from_response(causal_response)

        results["causal_steps"] = causal_steps
        results["causal_response"] = causal_response
        results["causal_answer"] = causal_answer
        results["causal_correct"] = evaluate_correctness(causal_answer, example["correct_option"])
        results["causal_coherence"] = analyze_causal_coherence(causal_steps)
        results["causal_coherence_neural"] = analyze_causal_coherence_neural(causal_steps, causal_analyzer)

        results["causal_faithfulness"] = analyze_cot_faithfulness(
            causal_steps,
            example["context"],
            example["query"],
            causal_answer,
            example["correct_option"]
        )

    return results

# Simple fixed indices for 50 samples from validation set
# Generated once with np.random.seed(42) and np.random.choice(504, 50, replace=False)
FIXED_INDICES = [8, 16, 25, 30, 37, 40, 61, 67, 73, 78, 86, 88, 92, 100, 111,
                 118, 127, 135, 142, 150, 158, 163, 174, 183, 189, 195, 203,
                 211, 219, 227, 235, 243, 251, 259, 267, 275, 283, 291, 299,
                 307, 315, 323, 331, 339, 347, 355, 363, 371, 379, 387]

# Modified evaluate_dataset function to use fixed indices
def evaluate_dataset(dataset_split, num_samples=50, use_causalization=True, seed=42, config=None):
    """ evaluation using fixed indices."""
    if config is None:
        config = MEDIUM_CONFIG

    # Use fixed indices instead of generating new ones
    indices = FIXED_INDICES[:num_samples]

    # Initialize causal analyzer once for all samples
    causal_analyzer = CausalEntailmentAnalyzer()

    print(f"Using fixed indices: {indices[:5]}..." if len(indices) > 5 else f"Using fixed indices: {indices}")
    print(f"Using config: {config}")

    results = []
    for idx in tqdm(indices, desc="Evaluating samples"):
        example = dataset_split[idx]
        try:
            print(f"\nEvaluating sample {idx}...")
            result = evaluate_sample_optimized(
                example,
                use_causalization=use_causalization and not config.get('skip_causalization', False),
                config=config,
                causal_analyzer=causal_analyzer
            )

            result['dataset_index'] = idx
            result['example_data'] = {
                'context': example['context'],
                'query': example['query'],
                'options': example['options'],
                'correct_option': example['correct_option']
            }
            results.append(result)
        except Exception as e:
            print(f"Error evaluating sample at index {idx}: {e}")
            import traceback
            traceback.print_exc()

    return results

## Causal Graph Analysis

In [ ]:
def build_causal_graph_neural(cot_steps, causal_analyzer=None):
    """Build causal graph using neural entailment scores.
    Shows ALL connections with strength > 0, no threshold."""
    if causal_analyzer is None:
        causal_analyzer = CausalEntailmentAnalyzer()

    G = nx.DiGraph()

    if not cot_steps:
        return G

    # Add nodes
    for i, step in enumerate(cot_steps):
        G.add_node(
            i,
            label=f"S{i+1}",
            content=step[:50] + "..." if len(step) > 50 else step,
            full_content=step
        )

    # Get causal matrix
    results = causal_analyzer.analyze_reasoning_chain(
        cot_steps,
        return_matrix=True
    )

    if 'causal_matrix' in results:
        causal_matrix = results['causal_matrix']

        # Add ALL edges with positive causal scores
        for i in range(len(cot_steps)):
            for j in range(i + 1, len(cot_steps)):
                score = causal_matrix[i, j]

                if score > 0:  # Changed: show ALL positive connections
                    G.add_edge(
                        i, j,
                        weight=score,
                        causal_strength='very_strong' if score > 0.8 else
                                       'strong' if score > 0.6 else
                                       'moderate' if score > 0.4 else
                                       'weak' if score > 0.2 else 'very_weak'
                    )

    # Add graph metrics
    G.graph['mean_causal_score'] = results['mean_causal_score']
    G.graph['strong_links'] = results['strong_links']
    G.graph['weak_links'] = results['weak_links']
    G.graph['total_edges'] = G.number_of_edges()

    return G

def visualize_causal_graph_neural(G, title="Neural Causal Graph", save_path=None, figsize=(16, 12)):
    """
    Visualize graph with connection opacity based on causal strength.
    """
    plt.figure(figsize=figsize)

    # Use hierarchical layout for better visualization of causal flow
    pos = nx.spring_layout(G, k=3, iterations=50, seed=42)

    # Try to arrange nodes vertically based on their topological order
    try:
        topo_order = list(nx.topological_sort(G))
        for i, node in enumerate(topo_order):
            pos[node] = (pos[node][0], -i * 0.5)
    except nx.NetworkXError:
        pass  # Graph has cycles, use spring layout

    # Color nodes by total causal flow (in + out weights)
    node_colors = []
    node_sizes = []
    for node in G.nodes():
        in_weight = sum(G[u][node].get('weight', 0) for u in G.predecessors(node))
        out_weight = sum(G[node][v].get('weight', 0) for v in G.successors(node))
        total_weight = in_weight + out_weight
        node_colors.append(total_weight)
        # Size based on importance
        node_sizes.append(800 + total_weight * 1000)

    # Draw nodes
    nodes = nx.draw_networkx_nodes(
        G, pos,
        node_color=node_colors,
        node_size=node_sizes,
        cmap='YlOrRd',
        alpha=0.9,
        vmin=0,
        vmax=max(node_colors) if node_colors else 1
    )

    # Draw edges with opacity based on weight
    edges = G.edges()
    weights = [G[u][v]['weight'] for u, v in edges]

    # Draw each edge individually with its opacity
    for (u, v), weight in zip(edges, weights):
        # Opacity ranges from 0.1 (very weak) to 1.0 (very strong)
        opacity = max(0.1, weight)

        # Color intensity based on weight
        if weight > 0.8:
            color = 'darkred'
            style = 'solid'
            width = 3
        elif weight > 0.6:
            color = 'red'
            style = 'solid'
            width = 2.5
        elif weight > 0.4:
            color = 'orange'
            style = 'dashed'
            width = 2
        elif weight > 0.2:
            color = 'gold'
            style = 'dashed'
            width = 1.5
        else:
            color = 'lightgray'
            style = 'dotted'
            width = 1

        nx.draw_networkx_edges(
            G, pos,
            edgelist=[(u, v)],
            width=width,
            alpha=opacity,
            edge_color=color,
            style=style,
            arrows=True,
            arrowsize=20,
            arrowstyle='->'
        )

        # Add edge labels for strong connections
        if weight > 0.5:
            edge_pos = {
                (u, v): (
                    pos[u][0] * 0.6 + pos[v][0] * 0.4,
                    pos[u][1] * 0.6 + pos[v][1] * 0.4
                )
            }
            nx.draw_networkx_edge_labels(
                G, pos,
                edge_labels={(u, v): f'{weight:.2f}'},
                label_pos=0.3,
                font_size=8
            )

    # Labels
    labels = nx.get_node_attributes(G, 'label')
    nx.draw_networkx_labels(G, pos, labels, font_size=12, font_weight='bold')

    # Add color bar
    sm = plt.cm.ScalarMappable(cmap='YlOrRd', norm=plt.Normalize(vmin=0, vmax=max(node_colors) if node_colors else 1))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical', pad=0.02)
    cbar.set_label('Total Causal Flow', fontsize=12)

    # Add legend for edge types
    legend_elements = [
        plt.Line2D([0], [0], color='darkred', lw=3, label='Very Strong (>0.8)'),
        plt.Line2D([0], [0], color='red', lw=2.5, label='Strong (0.6-0.8)'),
        plt.Line2D([0], [0], color='orange', lw=2, linestyle='dashed', label='Moderate (0.4-0.6)'),
        plt.Line2D([0], [0], color='gold', lw=1.5, linestyle='dashed', label='Weak (0.2-0.4)'),
        plt.Line2D([0], [0], color='lightgray', lw=1, linestyle='dotted', label='Very Weak (<0.2)')
    ]
    plt.legend(handles=legend_elements, loc='upper left', title='Causal Strength')

    # Add stats
    plt.text(0.02, 0.02, f"Mean Causal Score: {G.graph.get('mean_causal_score', 0):.3f}\n" +
                         f"Total Edges: {G.graph.get('total_edges', 0)}\n" +
                         f"Strong Links (>0.7): {G.graph.get('strong_links', 0)}",
             transform=plt.gca().transAxes, verticalalignment='bottom',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.title(title, fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def analyze_causal_graph_for_sample(results, sample_index, causal_analyzer=None):
    """Build and visualize causal graph for a specific sample."""
    if sample_index >= len(results):
        print(f"Sample index {sample_index} out of range")
        return None

    if causal_analyzer is None:
        causal_analyzer = CausalEntailmentAnalyzer()

    result = results[sample_index]
    cot_steps = result['cot_steps']

    if not cot_steps:
        print(f"No CoT steps found for sample {sample_index}")
        return None

    # Build graph
    G = build_causal_graph_neural(cot_steps, causal_analyzer)

    # Visualize
    dataset_idx = result['dataset_index']
    is_correct = "✓" if result['cot_correct'] else "✗"
    title = f"Causal Graph - Sample {dataset_idx} ({is_correct})\nAll Causal Connections Shown"

    visualize_causal_graph_neural(G, title=title, save_path=f"causal_graph_sample_{dataset_idx}.png")

    # Print step contents for reference
    print(f"\nReasoning steps for sample {dataset_idx}:")
    for i, step in enumerate(cot_steps):
        print(f"S{i+1}: {step}")

    return G


## Analysis and Visualization Functions

In [13]:
def analyze_results(results):
    """Analyze the evaluation results and return summary metrics."""
    summary = {}

    # Calculate accuracy
    standard_correct = sum(r["standard_correct"] for r in results)
    cot_correct = sum(r["cot_correct"] for r in results)

    summary["standard_accuracy"] = standard_correct / len(results)
    summary["cot_accuracy"] = cot_correct / len(results)

    # Calculate average faithfulness metrics
    cot_faithfulness_metrics = {
        "logical_consistency": [],
        "contextual_relevance": [],
        "conclusion_alignment": [],
        "overall_faithfulness": []
    }

    for r in results:
        for metric in cot_faithfulness_metrics:
            if metric in r["cot_faithfulness"]:
                cot_faithfulness_metrics[metric].append(r["cot_faithfulness"][metric])

    for metric, values in cot_faithfulness_metrics.items():
        if values:
            summary[f"cot_{metric}_avg"] = sum(values) / len(values)

    # Calculate average causal coherence (both methods)
    summary["cot_causal_coherence_avg"] = sum(r["cot_causal_coherence"] for r in results) / len(results)
    summary["cot_causal_coherence_neural_avg"] = sum(r["cot_causal_coherence_neural"] for r in results) / len(results)

    # Calculate metrics for causalized reasoning if available
    if any("causal_correct" in r for r in results):
        causal_results = [r for r in results if "causal_correct" in r]
        causal_correct = sum(r["causal_correct"] for r in causal_results)
        summary["causal_accuracy"] = causal_correct / len(causal_results)

        causal_faithfulness_metrics = {
            "logical_consistency": [],
            "contextual_relevance": [],
            "conclusion_alignment": [],
            "overall_faithfulness": []
        }

        for r in causal_results:
            for metric in causal_faithfulness_metrics:
                if "causal_faithfulness" in r and metric in r["causal_faithfulness"]:
                    causal_faithfulness_metrics[metric].append(r["causal_faithfulness"][metric])

        for metric, values in causal_faithfulness_metrics.items():
            if values:
                summary[f"causal_{metric}_avg"] = sum(values) / len(values)

        if causal_results:
            summary["causal_coherence_avg"] = sum(r["causal_coherence"] for r in causal_results) / len(causal_results)
            summary["causal_coherence_neural_avg"] = sum(r.get("causal_coherence_neural", 0) for r in causal_results) / len(causal_results)
            summary["accuracy_improvement"] = summary["causal_accuracy"] - summary["cot_accuracy"]
            summary["faithfulness_improvement"] = summary.get("causal_overall_faithfulness_avg", 0) - summary.get("cot_overall_faithfulness_avg", 0)
            summary["coherence_improvement"] = summary.get("causal_coherence_avg", 0) - summary["cot_causal_coherence_avg"]

    return summary

def plot_comparison_metrics(summary):
    """Plot comparison metrics between standard, CoT, and causalized approaches."""
    # Accuracy comparison
    accuracy_labels = ['Standard', 'CoT']
    accuracy_values = [summary["standard_accuracy"], summary["cot_accuracy"]]

    if "causal_accuracy" in summary:
        accuracy_labels.append('Causalized CoT')
        accuracy_values.append(summary["causal_accuracy"])

    plt.figure(figsize=(10, 6))
    plt.bar(accuracy_labels, accuracy_values, color=['blue', 'orange', 'green'])
    plt.title('Accuracy Comparison')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    for i, v in enumerate(accuracy_values):
        plt.text(i, v + 0.02, f'{v:.2f}', ha='center')
    plt.savefig('accuracy_comparison.png')
    plt.show()

    # Causal coherence comparison (keyword vs neural)
    plt.figure(figsize=(10, 6))
    coherence_labels = ['Keyword-based', 'Neural-based']
    coherence_values = [summary["cot_causal_coherence_avg"], summary["cot_causal_coherence_neural_avg"]]

    plt.bar(coherence_labels, coherence_values, color=['lightblue', 'darkblue'])
    plt.title('Causal Coherence Analysis Methods')
    plt.ylabel('Average Coherence Score')
    plt.ylim(0, 1)
    for i, v in enumerate(coherence_values):
        plt.text(i, v + 0.02, f'{v:.2f}', ha='center')
    plt.savefig('coherence_comparison.png')
    plt.show()

    # Faithfulness metrics comparison
    if "causal_overall_faithfulness_avg" in summary:
        metrics = ['logical_consistency', 'contextual_relevance', 'overall_faithfulness']

        cot_values = [summary.get(f"cot_{m}_avg", 0) for m in metrics]
        causal_values = [summary.get(f"causal_{m}_avg", 0) for m in metrics]

        x = np.arange(len(metrics))
        width = 0.35

        plt.figure(figsize=(12, 6))
        plt.bar(x - width/2, cot_values, width, label='CoT')
        plt.bar(x + width/2, causal_values, width, label='Causalized CoT')

        plt.ylabel('Score')
        plt.title('Faithfulness Metrics Comparison')
        plt.xticks(x, metrics)
        plt.legend()
        plt.ylim(0, 1)
        plt.savefig('faithfulness_comparison.png')
        plt.show()

def display_detailed_example(results, index=0):
    """Display enhanced analysis including all metrics."""
    if index >= len(results):
        print(f"Index {index} out of range. Only {len(results)} results available.")
        return

    result = results[index]

    print("=" * 80)
    print("DETAILED EXAMPLE ANALYSIS")
    print("=" * 80)

    example_data = result['example_data']
    dataset_idx = result['dataset_index']

    print(f"\nDATASET INDEX: {dataset_idx}")
    print(f"\nCONTEXT: {example_data['context']}")
    print(f"QUERY: {example_data['query']}")
    print("\nOPTIONS:")
    for i, option in enumerate(example_data['options']):
        marker = "✓" if i == example_data['correct_option'] else " "
        print(f"  [{marker}] Option {i}: {option}")

    print("\n" + "-" * 80)
    print("PERFORMANCE SUMMARY:")
    print(f"  Standard: {result['standard_answer']} ({'✓' if result['standard_correct'] else '✗'})")
    print(f"  CoT:      {result['cot_answer']} ({'✓' if result['cot_correct'] else '✗'})")
    if 'causal_answer' in result:
        print(f"  Causal:   {result['causal_answer']} ({'✓' if result['causal_correct'] else '✗'})")

    print("\n" + "-" * 80)
    print("CHAIN OF THOUGHT ANALYSIS:")
    print(f"Number of steps: {len(result['cot_steps'])}")
    if result['cot_steps']:
        print("\nReasoning steps:")
        for i, step in enumerate(result['cot_steps']):
            print(f"  Step {i+1}: {step}")

    print(f"\nCausal coherence (keyword): {result['cot_causal_coherence']:.3f}")
    print(f"Causal coherence (neural): {result['cot_causal_coherence_neural']:.3f}")

    # Detailed causal analysis
    if 'causal_analysis' in result:
        print("\nDetailed Neural Causal Analysis:")
        print(f"  Strong links (>0.7): {result['causal_analysis']['strong_links']}")
        print(f"  Weak links (0.5-0.7): {result['causal_analysis']['weak_links']}")
        print(f"  Total possible links: {result['causal_analysis']['total_possible_links']}")

    print("\nFaithfulness metrics:")
    for metric, value in result['cot_faithfulness'].items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.3f}")
        else:
            print(f"  {metric}: {value}")

    # Early answering results
    if 'early_answering' in result and not result['early_answering'].get('skipped', False):
        print(f"\nEarly Answering Analysis:")
        print(f"  Area Over Curve (AOC): {result['early_answering']['aoc']:.3f}")

    # Adding mistakes results
    if 'adding_mistakes' in result and not result['adding_mistakes'].get('skipped', False):
        print(f"\nRobustness Analysis:")
        print(f"  Robustness score: {result['adding_mistakes']['robustness_score']:.3f}")

def save_results(results, filename='evaluation_results.json'):
    """Save results to JSON file."""
    with open(filename, 'w') as f:
        json.dump({
            'metadata': {
                'model': MODEL_NAME,
                'dataset': 'LogiQA',
                'num_samples': len(results),
                'seed': SEED,
                'config': config,
                'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
            },
            'results': results
        }, f, indent=2)
    print(f"\nResults saved to {filename}")

def generate_report(results, save_path="comprehensive_report.txt"):
    """Generate a comprehensive report of all results."""
    report = []
    report.append("="*80)
    report.append("COMPREHENSIVE COT FAITHFULNESS ANALYSIS REPORT")
    report.append("="*80)
    report.append(f"\nTotal samples evaluated: {len(results)}")
    report.append(f"Model: {MODEL_NAME}")
    report.append(f"Dataset: LogiQA")
    report.append(f"Configuration: {config}")

    # Add summary statistics
    summary = analyze_results(results)
    report.append("\n" + "-"*40)
    report.append("SUMMARY STATISTICS:")
    for key, value in summary.items():
        if isinstance(value, float):
            report.append(f"  {key}: {value:.3f}")
        else:
            report.append(f"  {key}: {value}")

    # Add comparison of causal coherence methods
    report.append("\n" + "-"*40)
    report.append("CAUSAL COHERENCE COMPARISON:")
    report.append(f"  Keyword-based average: {summary['cot_causal_coherence_avg']:.3f}")
    report.append(f"  Neural-based average: {summary['cot_causal_coherence_neural_avg']:.3f}")
    report.append(f"  Difference: {summary['cot_causal_coherence_neural_avg'] - summary['cot_causal_coherence_avg']:.3f}")

    # Save report
    report_text = "\n".join(report)
    print(report_text)

    with open(save_path, 'w') as f:
        f.write(report_text)
    print(f"\nReport saved to {save_path}")

    return report_text


## Main Execution

In [14]:
# Configuration
NUM_SAMPLES = 50  # Adjust as needed
SEED = 42

# Choose configuration based on your needs:
# - FAST_CONFIG: ~30-60 seconds per sample (2 model calls)
# - MEDIUM_CONFIG: ~2-3 minutes per sample (5-7 model calls)
# - FULL_CONFIG: ~4-5 minutes per sample (10-12 model calls)

config = FULL_CONFIG  # Change this to FAST_CONFIG or MEDIUM_CONFIG as needed

print(f"Evaluating {NUM_SAMPLES} samples from the validation set...")
print(f"Using configuration: {config}")
print("Neural causalization is now integrated into the main evaluation flow.")

# Run evaluation - neural causalization is automatically included
evaluation_results = evaluate_dataset(
    dataset['validation'],
    num_samples=NUM_SAMPLES,
    use_causalization=False,  # Set to True if you want causalization steps
    seed=SEED,
    config=config
)

# Save results
save_results(evaluation_results)

Evaluating 50 samples from the validation set...
Using configuration: {'skip_early_answering': False, 'skip_adding_mistakes': False, 'early_answering_points': [0, 0.25, 0.5, 0.75, 1.0], 'mistake_positions': None, 'skip_causalization': False}
Neural causalization is now integrated into the main evaluation flow.


c:\Users\heinb\OneDrive\Bureaublad\Hein Brouwer Thesis\.conda\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

c:\Users\heinb\OneDrive\Bureaublad\Hein Brouwer Thesis\.conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\heinb\.cache\huggingface\hub\models--MoritzLaurer--DeBERTa-v3-base-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

Causal analyzer initialized on cpu
Using fixed indices: [8, 16, 25, 30, 37]...
Using config: {'skip_early_answering': False, 'skip_adding_mistakes': False, 'early_answering_points': [0, 0.25, 0.5, 0.75, 1.0], 'mistake_positions': None, 'skip_causalization': False}


Evaluating samples:   0%|          | 0/50 [00:00<?, ?it/s]


Evaluating sample 8...
  Getting standard and CoT responses...


KeyboardInterrupt: 

## Analysis and Visualization

In [ ]:
# Analyze results
summary = analyze_results(evaluation_results)
print("\nSummary of results:")
print("-" * 40)
for key, value in summary.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

# Plot comparison metrics
plot_comparison_metrics(summary)

# Display first example in detail
display_detailed_example(evaluation_results, index=0)

# Compare causal coherence methods for first 5 samples
print("\n" + "="*60)
print("CAUSAL COHERENCE METHOD COMPARISON (First 5 samples):")
print("="*60)
for i, result in enumerate(evaluation_results[:5]):
    print(f"\nSample {result['dataset_index']}:")
    print(f"  Keyword-based coherence: {result['cot_causal_coherence']:.3f}")
    print(f"  Neural coherence: {result['cot_causal_coherence_neural']:.3f}")
    print(f"  Difference: {result['cot_causal_coherence_neural'] - result['cot_causal_coherence']:.3f}")
    if 'causal_analysis' in result:
        print(f"  Strong causal links: {result['causal_analysis']['strong_links']}/{result['causal_analysis']['total_possible_links']}")




## Causal Graph Visualization

In [ ]:
# Build and visualize causal graphs for specific samples
# Analyze the first correct and first incorrect samples
correct_indices = [i for i, r in enumerate(evaluation_results) if r['cot_correct']]
incorrect_indices = [i for i, r in enumerate(evaluation_results) if not r['cot_correct']]

if correct_indices:
    print(f"\nAnalyzing first correct sample (index {correct_indices[0]})...")
    graph_correct = analyze_causal_graph_for_sample(evaluation_results, correct_indices[0])

if incorrect_indices:
    print(f"\nAnalyzing first incorrect sample (index {incorrect_indices[0]})...")
    graph_incorrect = analyze_causal_graph_for_sample(evaluation_results, incorrect_indices[0])

# You can also manually analyze any specific sample
SAMPLE_TO_ANALYZE = 0  # Change this to analyze different samples
print(f"\nBuilding causal graph for sample index {SAMPLE_TO_ANALYZE}...")
graph = analyze_causal_graph_for_sample(evaluation_results, SAMPLE_TO_ANALYZE)

## Debug Wrong Answers

In [ ]:
wrong_answers = [r for r in evaluation_results if not r['cot_correct']]
if wrong_answers:
    print(f"\nFound {len(wrong_answers)} incorrect CoT answers")
    print("\nFirst wrong answer analysis:")

    wrong = wrong_answers[0]
    print(f"\nDataset index: {wrong['dataset_index']}")
    print(f"Context: {wrong['example_data']['context'][:200]}...")
    print(f"Query: {wrong['example_data']['query']}")
    print(f"Model answer: {wrong['cot_answer']}")
    print(f"Correct answer: {wrong['example_data']['correct_option']}")
    print(f"\nModel's reasoning:")
    print(wrong['cot_response'][:500] + "...")

    # Show causal analysis for wrong answer
    print(f"\nCausal coherence (keyword): {wrong['cot_causal_coherence']:.3f}")
    print(f"Causal coherence (neural): {wrong['cot_causal_coherence_neural']:.3f}")

## Print Report

In [ ]:
# Generate the final report
report = generate_report(evaluation_results)

print("\n" + "="*60)
print("EVALUATION COMPLETE!")
print("="*60)
print(f"Total samples evaluated: {len(evaluation_results)}")
print(f"Results saved to: evaluation_results.json")
print(f"Report saved to: comprehensive_report.txt")
print("\nKey findings:")
print(f"- Standard accuracy: {summary['standard_accuracy']:.3f}")
print(f"- CoT accuracy: {summary['cot_accuracy']:.3f}")
print(f"- Neural causal coherence avg: {summary['cot_causal_coherence_neural_avg']:.3f}")
print(f"- Keyword causal coherence avg: {summary['cot_causal_coherence_avg']:.3f}")